# Narrative Embeddings
In this file, I will embedd the key narrratives, and look at the cosine similarity to the quotes, and the themes I extracted from his speeches. 

In [21]:
import pandas as pd
import numpy as np
import os
import re
import json
import time
from openai import OpenAI
from dotenv import load_dotenv
from tqdm.auto import tqdm
# Configure tqdm for pandas
tqdm.pandas()

# Load environment variables from .env file
load_dotenv()

True

In [7]:
# Access your API key
openai_api_key = os.getenv("OpenAI_Key_CSS")

# Initialize OpenAI client
client = OpenAI(api_key=openai_api_key)

## Get Embeddings of Arguments/Narratives
First, we have to get the embeddings of the key themes we are working with. 

In [4]:
#read the initial arguments csv file
df_args = pd.read_csv('Data/Arguments_Embeddigns/Campaing_Promises_embeddings_p1.csv')
df_args.head()


,Label,Description
0,Economy and Jobs,Trump's economic promises center around revita...
1,Border Security and Immigration,Trump places significant emphasis on border se...
2,Energy Policy,Trump's energy platform focuses on achieving A...
3,Law and Order,Trump's law and order platform emphasizes supp...
4,Education Reform,Trump's education platform focuses on limiting...


In [9]:
def get_embedding(text, model="text-embedding-3-large"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding


In [10]:
df_args['embedding'] = df_args['Description'].progress_apply(lambda x: get_embedding(x))

100%|██████████| 11/11 [00:07<00:00,  1.46it/s]


In [11]:
df_args.head()

,Label,Description,embedding
0,Economy and Jobs,Trump's economic promises center around revita...,"[-0.008811792358756065, 0.00041153558413498104..."
1,Border Security and Immigration,Trump places significant emphasis on border se...,"[-0.012985062785446644, -0.03436608985066414, ..."
2,Energy Policy,Trump's energy platform focuses on achieving A...,"[-0.014339656569063663, -0.02485981024801731, ..."
3,Law and Order,Trump's law and order platform emphasizes supp...,"[-0.03602679818868637, -0.05606026202440262, -..."
4,Education Reform,Trump's education platform focuses on limiting...,"[-0.019362250342965126, -0.029165612533688545,..."


In [12]:
df_args.to_csv('Data/Arguments_Embeddigns/Campaing_Promises_embeddings_p1.csv', index=False)
#save as feather
df_args.to_feather('Data/Arguments_Embeddigns/Campaing_Promises_embeddings_p1.feather')
#save as parquet
df_args.to_parquet('Data/Arguments_Embeddigns/Campaing_Promises_embeddings_p1.parquet')

## embedding descriptions

In [15]:
promises_df = pd.read_parquet('Data/Promises_From_Speech/promises_p1.parquet')
print(promises_df.shape)
promises_df.head()

(1803, 6)


,video_id,Label,Description,Excerpt,Issue,Importance
0,0npi0hevtKg,Secure the Border,Trump promises to secure the U.S. border by co...,I'm going to be a dictator for one day and we'...,Illegal immigration and border security,9
1,0npi0hevtKg,Energy Independence,Trump pledges to make the U.S. energy independ...,we're going to become energy independent we're...,Energy independence and economic stability,8
2,0npi0hevtKg,Military Strength,Trump promises to strengthen the U.S. military...,our military has gone woke and we are teeterin...,Military readiness and national security,7
3,0npi0hevtKg,Economic Revival,Trump aims to revive the U.S. economy by reduc...,our middle class is being crushed by Joe Biden...,Economic stability and inflation,8
4,0npi0hevtKg,Law and Order,Trump promises to restore law and order by sup...,we're going to give the police back their Auth...,Crime and public safety,7


In [17]:
promises_df['embedding_description'] = promises_df['Description'].progress_apply(lambda x: get_embedding(x))
print(promises_df[promises_df['Description'].isna()]['Description'])
promises_df['embedding_description'].head()

100%|██████████| 1803/1803 [22:41<00:00,  1.32it/s]  

Series([], Name: Description, dtype: object)


0    [0.0071108583360910416, -0.03017244301736355, ...
1    [-0.008141732774674892, -0.0009867080952972174...
2    [-0.043753284960985184, -0.003683375660330057,...
3    [-0.025632129982113838, -0.01909560151398182, ...
4    [-0.014920054003596306, -0.05041293427348137, ...
Name: embedding_description, dtype: object

In [18]:
#check if there are any null values in the embedding column
print(promises_df['embedding_description'].isna().sum())

0


In [20]:
#save to parquet, feather and csv
promises_df.to_csv('Data/Arguments_Embeddigns/promises_p1_embeddings.csv', index=False)
promises_df.to_feather('Data/Arguments_Embeddigns/promises_p1_embeddings.feather')
promises_df.to_parquet('Data/Arguments_Embeddigns/promises_p1_embeddings.parquet')

Now I have the embeddings of the key descriptions saved, so I can move on getting cosine similarities for them with the embedded narratives. 